In [1]:
!pip install transformers datasets

   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
    --------------------------------------- 0.5/25.1 MB 3.4 MB/s eta 0:00:08
   -- ------------------------------------- 1.3/25.1 MB 3.5 MB/s eta 0:00:07
   --- ------------------------------------ 2.1/25.1 MB 4.1 MB/s eta 0:00:06
   ---- ----------------------------------- 2.9/25.1 MB 3.8 MB/s eta 0:00:06
   ------ --------------------------------- 3.9/25.1 MB 3.9 MB/s eta 0:00:06
   ------- -------------------------------- 5.0/25.1 MB 4.1 MB/s eta 0:00:05
   --------- ------------------------------ 6.0/25.1 MB 4.3 MB/s eta 0:00:05
   ----------- ---------------------------- 7.3/25.1 MB 4.4 MB/s eta 0:00:05
   ------------- -------------------------- 8.4/25.1 MB 4.4 MB/s eta 0:00:04
   --------------- ------------------------ 9.4/25.1 MB 4.6 MB/s eta 0:00:04
   ----------------- ---------------------- 10.7/25.1 MB 4.7 MB/s eta 0:00:04
   ------------------ --------------------- 11.8/25.1 MB 4.7 MB/s eta 0:00:03
   -

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments

c:\Users\dharane\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load the dataset
data_path = '/content/drive/MyDrive/bio_summary_final.csv' 
df = pd.read_csv(data_path)

In [6]:
# Convert the DataFrame to a Hugging Face Dataset format
dataset = Dataset.from_pandas(df)

In [7]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
# Tokenization function with padding
def preprocess_data(examples):
    inputs = ["summarize: " + doc for doc in examples["longtext"]]
    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True)  # Adding padding

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=150, padding="max_length", truncation=True)  # Adding padding

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
print(df.columns)

Index(['longtext', 'summary'], dtype='object')


In [ ]:
# Apply preprocessing to the dataset
tokenized_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [11]:
# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./flan_t5_finetuned",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,  
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    push_to_hub=False
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset, 
)

In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.628453
2,No log,2.559380
3,No log,2.540879


TrainOutput(global_step=375, training_loss=2.9246923828125, metrics={'train_runtime': 436.3881, 'train_samples_per_second': 3.43, 'train_steps_per_second': 0.859, 'total_flos': 1025081756614656.0, 'train_loss': 2.9246923828125, 'epoch': 3.0})

In [15]:
model.save_pretrained('/content/drive/MyDrive/flan_t5_finetuned_model')
tokenizer.save_pretrained('/content/drive/MyDrive/flan_t5_finetuned_model')

('/content/drive/MyDrive/flan_t5_finetuned_model/tokenizer_config.json',
 '/content/drive/MyDrive/flan_t5_finetuned_model/special_tokens_map.json',
 '/content/drive/MyDrive/flan_t5_finetuned_model/spiece.model',
 '/content/drive/MyDrive/flan_t5_finetuned_model/added_tokens.json',
 '/content/drive/MyDrive/flan_t5_finetuned_model/tokenizer.json')

In [ ]:
# Define the path 
model_save_path = '/content/drive/MyDrive/flan_t5_finetuned_model_2nd' 

In [17]:
# Save the model
trainer.model.save_pretrained(model_save_path)

In [18]:
# Save the tokenizer
tokenizer.save_pretrained(model_save_path)

('/content/drive/MyDrive/flan_t5_finetuned_model_2nd/tokenizer_config.json',
 '/content/drive/MyDrive/flan_t5_finetuned_model_2nd/special_tokens_map.json',
 '/content/drive/MyDrive/flan_t5_finetuned_model_2nd/spiece.model',
 '/content/drive/MyDrive/flan_t5_finetuned_model_2nd/added_tokens.json',
 '/content/drive/MyDrive/flan_t5_finetuned_model_2nd/tokenizer.json')